### Preamble

In [1]:
# Configure libraries

%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score
import geohash
from sklearn.metrics import mean_squared_error
import random

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# Funtion for cross-validation over a grid of parameters

def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None, verbose=0):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func, verbose=verbose)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds, verbose=verbose)
    gs.fit(X, y)
    print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_, gs.scorer_
    print "Best score: ", gs.best_score_
    best = gs.best_estimator_
    return best

###  Reading the data

For a Geohash precision of 6 (geohash length of 6 characters), we have roughly 300,000 records. For a precision of 7, we have about 4 million records.

In [3]:
# Each line is of the format:
# ((time_cat, time_num, time_cos, time_sin, day_cat, day_num, day_cos, day_sin, weekend, geohash), number of pickups)
names = ["time_cat", "time_num", "time_cos", "time_sin", "day_cat", "day_num", "day_cos", "day_sin", "weekend", "geohash", "pickups"]
dftaxi=pd.read_csv("./data/trainset7.csv", header=None, names = names)
print dftaxi.shape

(695424, 11)


### Define training and test sets

In [4]:
itrain, itest = train_test_split(xrange(dftaxi.shape[0]), train_size=0.8)
mask=np.ones(dftaxi.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)
mask[:10]

array([False, False,  True,  True, False,  True, False,  True,  True,
        True])

### Final preperation for machine learning

I tried initially doing one-hot-encoding on the geohashes, but I quickly realized that this was not feasible from a memory perspective. 3 million records times 100,000 features would not fit in memory. So I decided to go for the numerical latitude and longitude route. Using a random forest, we can easily detect higher order structures in these two variables.

In [5]:
# Split off the features
Xnames = ["time_cat", "time_num", "time_cos", "time_sin", "day_cat",
          "day_num", "day_cos", "day_sin", "weekend", "geohash"]
X = dftaxi[Xnames]

# Split off the target (which will be the logarithm of the number of pickups (+1))
y = np.log10(dftaxi['pickups']+1)

In [6]:
# Get the longitude and latitude from the geohash
def decodegeo(geo, which):
    if len(geo) >= 6:
        geodecoded = geohash.decode(geo)
        return geodecoded[which]
    else:
        return 0
X['latitude'] = X['geohash'].apply(lambda geo: decodegeo(geo, 0))
X['longitude'] = X['geohash'].apply(lambda geo: decodegeo(geo, 1))
X.head()

time_cat  time_num  time_cos  time_sin   day_cat   day_num   day_cos   day_sin  weekend  geohash   latitude  longitude
2013 10 10    22:30  0.947917  0.946930 -0.321439  Thursday  0.563988 -0.920261 -0.391305        0  dr72jhs  40.806656 -73.953781
     11 1     13:30  0.572917 -0.896873 -0.442289    Friday  0.653274 -0.571021 -0.820936        0  dr5rveb  40.758591 -73.937302
        17    00:00  0.010417  0.997859  0.065403    Sunday  0.858631  0.630773 -0.775968        1  dr72r34  40.831375 -73.857651
     9  17    23:00  0.968750  0.980785 -0.195090   Tuesday  0.281250 -0.195090  0.980785        0  dr72j6z  40.797043 -73.938675
     12 26    17:00  0.718750 -0.195090 -0.980785  Thursday  0.531250 -0.980785 -0.195090        0  dr72r1r  40.832748 -73.861771

In [7]:
# Create indicator variables for the hours and days of the week and drop the categorical values
# g = 5
# X = X.join(pd.get_dummies(X['time_cat']))\
#      .join(pd.get_dummies(X['day_cat']))\
#      .drop(['time_cat','day_cat','geohash'], axis=1)
X = X.drop(['time_cat','day_cat','geohash'], axis=1)
#     .join(pd.get_dummies(X['geohash'].str[:g]))\

### Get the training and test sets

In [8]:
Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]



In [9]:
n_samples = Xtrain.shape[0]
n_features = Xtrain.shape[1]
print Xtrain.shape


(556339, 9)


### Random Forest Regression

In [10]:
# Create a Random Forest Regression estimator
estimator = RandomForestRegressor(n_estimators=2, n_jobs=-1)

The step below takes about 25 mins on my laptop for 300,000 records and up to 100 estimators.

In [11]:
%%time
# Define a grid of parameters over which to optimize the random forest
# We will figure out which number of trees is optimal
parameters = {"n_estimators": [3],
              "max_features": ["auto"], # ["auto","sqrt","log2"]
              "max_depth": [3]}
best = cv_optimize(estimator, parameters, Xtrain, ytrain, n_folds=2, score_func='mean_squared_error', verbose=3)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] max_features=auto, n_estimators=3, max_depth=3 ..................


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV]  max_features=auto, n_estimators=3, max_depth=3, score=-0.014398 -   1.8s
[CV] max_features=auto, n_estimators=3, max_depth=3 ..................


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.5s finished


[CV]  max_features=auto, n_estimators=3, max_depth=3, score=-0.014356 -   1.7s
BEST {'max_features': 'auto', 'n_estimators': 3, 'max_depth': 3} -0.0143772083284 [mean: -0.01438, std: 0.00002, params: {'max_features': 'auto', 'n_estimators': 3, 'max_depth': 3}] make_scorer(mean_squared_error, greater_is_better=False)
Best score:  -0.0143772083284
CPU times: user 6.21 s, sys: 528 ms, total: 6.74 s
Wall time: 5.99 s


### Evaluate the results

In [12]:
# Fit the best Random Forest and calculate R^2 values for training and test sets
reg=best.fit(Xtrain, ytrain)
training_accuracy = reg.score(Xtrain, ytrain)
test_accuracy = reg.score(Xtest, ytest)
print "############# based on standard predict ################"
print "R^2 on training data: %0.4f" % (training_accuracy)
print "R^2 on test data:     %0.4f" % (test_accuracy)

############# based on standard predict ################
R^2 on training data: 0.0693
R^2 on test data:     0.0668


Sanity check on some records.

In [13]:
# Show some of the predictions vs. the real number of pickups
# predictions vs. real number of pickups
np.round(np.power(10,np.column_stack((reg.predict(Xtest),ytest))) - 1,decimals=0).astype(int)

array([[1, 2],
       [1, 1],
       [1, 1],
       ...,
       [1, 1],
       [1, 1],
       [1, 1]])

In [14]:
# Calculate the Root Mean Squared Error
rmse = np.sqrt(mean_squared_error(reg.predict(Xtest),ytest))
print "RMSE = %0.3f (this is in log-space!)" % rmse
print "So two thirds of the records would be a factor of less than %0.2f away from the real value." % np.power(10,rmse)

RMSE = 0.120 (this is in log-space!)
So two thirds of the records would be a factor of less than 1.32 away from the real value.


In [15]:
# What are the most important features?
import operator
dict_feat_imp = dict(zip(list(X.columns.values),reg.feature_importances_))

sorted_features = sorted(dict_feat_imp.items(), key=operator.itemgetter(1), reverse=True)
sorted_features

[('latitude', 0.5951082898372474),
 ('longitude', 0.40489171016275255),
 ('time_num', 0.0),
 ('day_num', 0.0),
 ('time_cos', 0.0),
 ('day_cos', 0.0),
 ('time_sin', 0.0),
 ('weekend', 0.0),
 ('day_sin', 0.0)]

### Create predictions for visualization

Now we are going to generate predictions that we can visualize in Tableau. We do this by generating all possible combinations of time and location so that we have a well filled space of predictions. Then we generate predictions for all these combinations and then export to .csv.

#### First we need a dataframe with all possible combinations of time and location

In [16]:
# Construct dataframes with all possible times (time_data) and all possible locations (loc_data)

# Columns about time
time_cols = list(X.columns.values)
time_cols.remove('latitude')
time_cols.remove('longitude')

# Columns about location
loc_cols = ['latitude', 'longitude']

# Unique times
time_data = X.drop(loc_cols, axis=1).drop_duplicates()

# In Tableau we are only going to look at Monday
time_data = time_data[time_data['day_num'] <= 1/7.]

# Unique locations
loc_data = X.drop(time_cols, axis=1).drop_duplicates()

# To reduce memory consumption in Tableau, we are only predicting for
# the region closely around Manhattan and the La Guardia and JFK airports
loc_data = loc_data[(loc_data['latitude'] > 40.5) & (loc_data['latitude'] < 41.1) &
                    (loc_data['longitude'] > -74.1) & (loc_data['longitude'] < -73.6)]

In [17]:
# Dummy column to be able to join them together
time_data['key'] = 1
loc_data['key'] = 1

# Merge the time_data and location_data
result = pd.merge(time_data, loc_data, on='key').drop(['key'], axis=1)
result = result[Xtrain.columns.values]

In [18]:
result.describe()


,time_num,time_cos,time_sin,day_num,day_cos,day_sin,weekend,latitude,longitude
count,681744.000000,6.817440e+05,6.817440e+05,681744.000000,681744.000000,681744.000000,681744,681744.000000,681744.000000
mean,0.500000,-3.401880e-17,-5.402986e-17,0.071429,0.871039,0.419470,0,40.750763,-73.901631
std,0.288613,7.071073e-01,7.071073e-01,0.041230,0.113335,0.229109,0,0.079246,0.062876
min,0.010417,-9.978589e-01,-9.978589e-01,0.001488,0.630773,0.009350,0,40.560837,-74.099350
25%,0.255208,-6.824693e-01,-6.824693e-01,0.036458,0.784712,0.227069,0,40.689926,-73.946915
50%,0.500000,-1.734723e-16,1.318390e-16,0.071429,0.900929,0.433865,0,40.742111,-73.907089
75%,0.744792,6.824693e-01,6.824693e-01,0.106399,0.973845,0.619808,0,40.821762,-73.859024
max,0.989583,9.978589e-01,9.978589e-01,0.141369,0.999956,0.775968,0,41.096420,-73.606339


In [19]:
#result = result.drop(['pred_pickups'], axis=1)
result.describe()

,time_num,time_cos,time_sin,day_num,day_cos,day_sin,weekend,latitude,longitude
count,681744.000000,6.817440e+05,6.817440e+05,681744.000000,681744.000000,681744.000000,681744,681744.000000,681744.000000
mean,0.500000,-3.401880e-17,-5.402986e-17,0.071429,0.871039,0.419470,0,40.750763,-73.901631
std,0.288613,7.071073e-01,7.071073e-01,0.041230,0.113335,0.229109,0,0.079246,0.062876
min,0.010417,-9.978589e-01,-9.978589e-01,0.001488,0.630773,0.009350,0,40.560837,-74.099350
25%,0.255208,-6.824693e-01,-6.824693e-01,0.036458,0.784712,0.227069,0,40.689926,-73.946915
50%,0.500000,-1.734723e-16,1.318390e-16,0.071429,0.900929,0.433865,0,40.742111,-73.907089
75%,0.744792,6.824693e-01,6.824693e-01,0.106399,0.973845,0.619808,0,40.821762,-73.859024
max,0.989583,9.978589e-01,9.978589e-01,0.141369,0.999956,0.775968,0,41.096420,-73.606339


#### Then we do the prediction

In [20]:
# Get the real number of pickups and take care that we can merge it with the predictions,
# by also taking the geohash and the timestamp
yy = dftaxi[['geohash','day_num','pickups']]



In [21]:

# Decode the geohash in the latitude and longitude
yy['latitude'] = yy['geohash'].apply(lambda geo: decodegeo(geo, 0))
yy['longitude'] = yy['geohash'].apply(lambda geo: decodegeo(geo, 1))



/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [22]:
# Do predictions and convert the logarithm to the normal numbers
result['pred_pickups'] = np.power(10,reg.predict(result)) - 1

In [27]:

result = result.sample(n=100)
result.shape

(100, 10)

In [28]:
yy = yy.sample (n=100)
yy.shape


(100, 5)

In [29]:

# Merge the predictions and the real pickups
result = pd.merge(result, yy, how='left', on=['day_num', 'latitude', 'longitude']).drop(['geohash'], axis=1)
result.head(10)

,time_num,time_cos,time_sin,day_num,day_cos,day_sin,weekend,latitude,longitude,pred_pickups,pickups
0,0.031250,0.980785,0.195090,0.004464,0.999607,0.028046,0,40.713272,-73.935928,1.332902,NaN
1,0.531250,-0.980785,-0.195090,0.075893,0.888446,0.458982,0,40.720139,-73.790359,2.031129,NaN
2,0.572917,-0.896873,-0.442289,0.081845,0.870662,0.491881,0,40.836868,-73.916702,1.049256,NaN
3,0.635417,-0.659346,-0.751840,0.090774,0.841713,0.539926,0,40.750351,-73.865891,1.332902,NaN
4,0.635417,-0.659346,-0.751840,0.090774,0.841713,0.539926,0,40.764084,-73.913956,1.332902,NaN
5,0.802083,0.321439,-0.946930,0.114583,0.751840,0.659346,0,40.724258,-73.937302,1.332902,NaN
6,0.552083,-0.946930,-0.321439,0.078869,0.879708,0.475515,0,40.724258,-74.001846,1.332902,NaN
7,0.114583,0.751840,0.659346,0.016369,0.994716,0.102669,0,40.613022,-73.942795,1.100890,NaN
8,0.385417,-0.751840,0.659346,0.055060,0.940754,0.339090,0,40.692673,-73.977127,1.100890,NaN
9,0.218750,0.195090,0.980785,0.031250,0.980785,0.195090,0,40.856094,-73.885117,1.049256,NaN


In [30]:
# Drop unnecessary columns to reduce memory consumption in Tableau
result = result.drop(['time_cos','day_num','time_sin','day_cos','day_sin','weekend'], axis=1)

In [31]:
# Write to csv
result.to_csv('./data/taxi-data-predictions_prec_7_monday.csv')